クロスデバイス　Geoとの併用

In [24]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

4646	東日本旅客鉄道株式会社

In [25]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190904


In [26]:
seed_table = "segment.geo_locations_mindist_geo_segment_jr_higashinihon_190827"
print(seed_table)

segment.geo_locations_mindist_geo_segment_jr_higashinihon_190827


In [27]:
uid_table = f"segment.xd_geo_locations_mindist_geo_segment_jr_higashinihon_{today_yymmdd}" #最終的にセグメントに使用するテーブル
print(uid_table)

segment.xd_geo_locations_mindist_geo_segment_jr_higashinihon_190904


In [28]:
xd_table = 'segment.drawbridge_device'

In [29]:
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    ,sd.place_name
    ,sd.min_dist
    ,sd.seg_info
    ,xd2.device
    from (select * from {seed_table} where (seg_info = '5' AND min_dist < 3) OR (seg_info IN ('1', '2', '3', '4') AND min_dist < 1)) sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in ('iphone'
    ,'android_phone'
    ,'windows_phone'
    ,'windows_computer'
    ,'linux_computer'
    ,'mac'
    ,'ipad'
    ,'android_tablet'
    ,'windows_tablet') 
    and sd.smn_uid <> xd2.smn_uid
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    #ImpalaにUp
    ir.upload_dataframe(uid_table, df_fix, ['bigint','string','double','string','string']) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [30]:
df_fix.shape[0]

7775646

In [32]:
df_fix.head()

,smn_uid,place_name,min_dist,seg_info,device
0,1776000020509332,新宿駅,2.717805,5,windows_computer
1,2223000594027682,新宿駅,2.110202,5,windows_computer
2,2224000732389296,銀座駅,0.573920,1,mac
3,2060000393112891,新宿駅,2.290118,5,android_phone
4,2198000144797922,新宿駅,1.773816,5,iphone
